<a href="https://colab.research.google.com/github/mmetawei/AFQC/blob/main/Report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print(" GENERATING FINAL PERFORMANCE REPORT")
print("="*70)

import json
from datetime import datetime
import zipfile
import os

# Create report data
report = {
    "project": "Quantum Circuit Error Mitigation AI Model Comparison",
    "generation_date": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "dataset_info": {
        "source": "quantum_circuit_data.csv",
        "total_samples": len(df),
        "features": X.shape[1],
        "target": target_col,
        "train_samples": len(X_train),
        "test_samples": len(X_test)
    },
    "performance_summary": {
        "best_model": results_df.iloc[0]['Model'],
        "best_r2": float(results_df.iloc[0]['R²']),
        "best_rmse": float(results_df.iloc[0]['RMSE']),
        "total_models_tested": len(results_df),
        "models_above_r2_0.9": len(results_df[results_df['R²'] > 0.9]),
        "models_above_r2_0.95": len(results_df[results_df['R²'] > 0.95])
    },
    "model_ranking": results_df.to_dict('records'),
    "key_findings": [
        "Tree-based models (Random Forest, XGBoost, LightGBM) perform exceptionally well for quantum noise prediction",
        "Gradient boosting algorithms achieve the highest accuracy",
        "Linear models struggle with complex quantum noise patterns",
        "Deep Neural Network shows competitive performance with proper tuning",
        "Ensemble methods provide marginal improvements over single models"
    ],
    "files_generated": [
        "model_comparison_results.csv",
        "first_model_random_forest.pkl",
        "scaler.pkl",
        "best_model.pkl",
        "dnn_model.h5",
        "voting_ensemble.pkl",
        "stacking_ensemble.pkl"
    ]
}

# Save report
with open('model_comparison_report.json', 'w') as f:
    json.dump(report, f, indent=2)

print(f" Report generated: 'model_comparison_report.json'")

# Create summary table
print("\n" + "="*80)
print(" FINAL MODEL COMPARISON SUMMARY")
print("="*80)

summary_df = results_df[['Model', 'R²', 'RMSE', 'MAE', 'Train Time (s)']].copy()
summary_df['R²'] = summary_df['R²'].apply(lambda x: f'{x:.6f}')
summary_df['RMSE'] = summary_df['RMSE'].apply(lambda x: f'{x:.6f}')
summary_df['MAE'] = summary_df['MAE'].apply(lambda x: f'{x:.6f}')
summary_df['Train Time (s)'] = summary_df['Train Time (s)'].apply(lambda x: f'{x:.2f}s' if pd.notnull(x) else 'N/A')

display(summary_df)

print("\n" + "="*80)
print(" KEY INSIGHTS & RECOMMENDATIONS")
print("="*80)

print(f"\n1.  BEST PERFORMER: {report['performance_summary']['best_model']}")
print(f"   R² Score: {report['performance_summary']['best_r2']:.6f}")
print(f"   Prediction Error (RMSE): {report['performance_summary']['best_rmse']:.6f}")

print(f"\n2.  OVERALL PERFORMANCE:")
print(f"   {report['performance_summary']['models_above_r2_0.95']} models achieved R² > 0.95 (Excellent)")
print(f"   {report['performance_summary']['models_above_r2_0.9']} models achieved R² > 0.9 (Very Good)")

print(f"\n3.  RECOMMENDATIONS:")
print(f"   • For production: Use {report['performance_summary']['best_model']}")
print(f"   • For interpretability: Use Random Forest (R² = {initial_rf_r2:.6f})")
print(f"   • For research: Explore Deep Neural Network further")
print(f"   • For speed: Consider LightGBM or XGBoost")

print(f"\n4.  NEXT STEPS:")
print(f"   • Hyperparameter tuning for top 3 models")
print(f"   • Feature importance analysis for interpretability")
print(f"   • Cross-validation to ensure robustness")
print(f"   • Deployment as API for real-time predictions")

# Create download package
print(f"\n Creating downloadable package...")
with zipfile.ZipFile('quantum_ai_models_package.zip', 'w') as zipf:
    for file in report['files_generated'] + ['model_comparison_report.json', 'quantum_circuit_data.csv']:
        if os.path.exists(file):
            zipf.write(file)

print(f" Download package created: 'quantum_ai_models_package.zip'")

# Provide download instructions
from google.colab import files

print(f"\n  DOWNLOAD INSTRUCTIONS:")
print(f"1. Complete package: files.download('quantum_ai_models_package.zip')")
print(f"2. Just results: files.download('model_comparison_results.csv')")
print(f"3. Best model: files.download('best_model.pkl')")
print(f"4. Full report: files.download('model_comparison_report.json')")

print(f"\n  COMPARISON COMPLETE! You've tested {len(results_df)} different AI models.")
